In [1]:
#Connect to GIS
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
import pandas as pd
import arcpy
from arcgis import features, GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from arcgis.features.analysis import join_features
from arcgis.features import FeatureLayerCollection, FeatureLayer
from arcgis.geometry import SpatialReference , Geometry
import json

In [3]:
output_gdb= r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"
directory = r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2"

In [4]:
# Fuzzy Site Analytics: ONLY fuzzypotentialsites and fuzzybuildableacreage
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"):
    out_raster = arcpy.sa.FuzzyOverlay(
        in_rasters="fuzzypotentialsites;FuzzyBuildableAcreage",
        overlay_type="SUM",
        gamma=0.9
    )
    out_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\fuzzySiteAnalytics_Validation")

In [5]:
# Fuzzy Composite with fuzzysiteanalytics_validation and UtilityAHJ_EC_Union_PolygonToRaster
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"):
    out_raster = arcpy.sa.FuzzyOverlay(
        in_rasters="fuzzySiteAnalytics_Validation;UtilityAHJ_EC_Union_PolygonToRaster",
        overlay_type="PRODUCT",
        gamma=0.9
    )
    out_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\fuzzycomposite_Validation1")

In [6]:
# Invert weighting scheme for UtilityAHJ_EC_Union

# Assign weights to Utility/EC combinations 1/4
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = 'Interstate Power and Light Company' And name = 'EC2024'",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    in_table="UtilityAHJ_EC_Union",
    field="weight",
    expression= "0.67",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'UtilityAHJ_EC_Union'>

In [7]:
# Assign weights to Utility/EC combinations 2/4
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = 'MidAmerican Energy' And name = 'EC2024'",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    in_table="UtilityAHJ_EC_Union",
    field="weight",
    expression= "0",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'UtilityAHJ_EC_Union'>

In [8]:
# Assign weights to Utility/EC combinations 3/4
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = 'Interstate Power and Light Company' And name = ''",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    in_table="UtilityAHJ_EC_Union",
    field="weight",
    expression="1",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'UtilityAHJ_EC_Union'>

In [9]:
# Assign weights to Utility/EC combinations 3/4
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = 'MidAmerican Energy' And name = ''",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    in_table="UtilityAHJ_EC_Union",
    field="weight",
    expression= "0.33",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'UtilityAHJ_EC_Union'>

In [10]:
#weighted overlay with inverse weights
arcpy.conversion.PolygonToRaster(
    in_features="UtilityAHJ_EC_Union",
    value_field="weight",
    out_rasterdataset=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\UtilityAHJ_ECUnion_Validation",
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=1400,
    build_rat="BUILD"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\UtilityAHJ_ECUnion_Validation'>

In [14]:
# Fuzzy Membership Inverse 1/3
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"):
    out_raster = arcpy.sa.FuzzyMembership(
        in_raster="BuildableAcreage",
        fuzzy_function="SMALL 44315.565 5",
        hedge="NONE"
    )
    out_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\fmBuildableAcreage_inverse")

In [15]:
# Fuzzy Membership Inverse 2/3
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"):
    out_raster = arcpy.sa.FuzzyMembership(
        in_raster="PotentialSites",
        fuzzy_function="SMALL 532.5 5",
        hedge="NONE"
    )
    out_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\fmPotentialSites_inverse")

In [16]:
# Fuzzy Membership Inverse 3/3
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"):
    out_raster = arcpy.sa.FuzzyMembership(
        in_raster="BuildableAcreage_PerSite",
        fuzzy_function="SMALL 102.175 5",
        hedge="NONE"
    )
    out_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\fmBuildableAcreagePerSite_inverse")

In [17]:
# Site Analytics Inverse Composite
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"):
    out_raster = arcpy.sa.FuzzyOverlay(
        in_rasters="fmPotentialSites_inverse;fmBuildableAcreagePerSite_inverse;fmBuildableAcreage_inverse",
        overlay_type="SUM",
        gamma=0.9
    )
    out_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\fuzzycomposite_inverse")

In [18]:
# Inverse site analytics composite and UtilityAHJ_ECUnion fuzzy overlay
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"):
    out_raster = arcpy.sa.FuzzyOverlay(
        in_rasters="fuzzycomposite_inverse;UtilityAHJ_ECUnion_Validation",
        overlay_type="PRODUCT",
        gamma=0.9
    )
    out_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\fuzzyunionanalytics_inverse")